In [137]:
from pynq import Overlay 
from pynq import MMIO
import time

overlay = Overlay('/home/xilinx/jupyter_notebooks/lab2/lab2_p4.bit')

base_address = 0x40000000
size = 0x40000
matrix_size = 16

offset_a = 0x0
offset_a2 = 0x0
offset_b = 0x2000
offset_b2 = 0x2000
offset_prod = 0x4000

mmio = MMIO(base_address,size)

In [138]:
matrix_mult = overlay.matrix_mult_0
matrix_mult.register_map


RegisterMap {
  CTRL = Register(AP_START=0, AP_DONE=0, AP_IDLE=1, AP_READY=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0, INTERRUPT=0, RESERVED_3=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED_0=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED_0=0)
}

In [139]:
mmio.write(0x0, 2)

In [140]:
mmio.read(0x0)

2

In [141]:
mmio.read(0x4)

0

In [142]:
def generate_random_matrices(rows, cols):
    a = np.random.randint(low=0, high=10, size=(rows, cols), dtype=np.int32)
    b = np.random.randint(low=0, high=10, size=(rows, cols), dtype=np.int32)
    return a, b

def generate_basic_matrices_value(rows, cols, value):
    a = np.full((rows, cols), value, dtype=np.int32)
    b = np.full((rows, cols), value, dtype=np.int32)
    return a, b 

def write_matrix_a(matrix):
    rows, cols = matrix.shape

    for row in range(rows):
        for col in range(cols):
            # offset for input a
            offset = offset_a + (row * cols + col) * 4
            
            # Get the value from the matrix
            value = int(matrix[row, col])
            
            # Write the value to the calculated offset
            mmio.write(offset, value)
            
def read_offset_a_matrix():
    output_matrix = [[0 for _ in range(matrix_size)] for _ in range(matrix_size)]
    for row in range(matrix_size):
        for col in range(matrix_size):
            # offset for input a
            offset = offset_a + (row * matrix_size + col) * 4
            
            # Get the value from the offset 
            value = mmio.read(offset)
            
            # Write the value to the calculated offset
            mmio.write(offset, value)
            
            # store value
            output_matrix[row][col] = value
    
    return output_matrix

def print_matrix(matrix):
    for row in matrix:
        print(" ".join(map(str, row)))
        
def start_calculation():
    matrix_mult = overlay.matrix_mult_0
    matrix_mult.register_map.CTRL.AP_START = 1
    
    # Wait for calculation to finish
    while matrix_mult.register_map.CTRL.AP_IDLE == 0: 
        pass
    
def read_offset_prod_matrix():
    output_matrix = [[0 for _ in range(8)] for _ in range(8)]
    for row in range(8):
        for col in range(8):
            # offset for input a
            offset = offset_prod + (row * 8 + col) * 4
            
            # Get the value from the offset 
            value = mmio.read(offset)
            
            # Write the value to the calculated offset
            mmio.write(offset, value)
            
            # store value
            output_matrix[row][col] = value
            
def matrix_multf(a, b):
    
    a = np.array(a, dtype=np.int32)
    b = np.array(b, dtype=np.int32)
    prod = np.zeros((len(a), len(a[0])),dtype=np.int32)
    
    IN_A_ROWS = len(a)
    IN_A_COLS = len(a[0])
    IN_B_ROWS = len(b)
    IN_B_COLS = len(b[0])
    
    
    for i in range(IN_A_ROWS):
        for j in range(IN_B_COLS):
            for k in range(IN_B_ROWS):
                prod[i][j] += a[i][k] * b[k][j]
    return prod

In [143]:
a, b = generate_basic_matrices_value(16,16,4)

write_matrix_a(a)
write_matrix_b(b)
x = read_offset_a_matrix()
y = read_offset_b_matrix()

print_matrix(x)
print()
print_matrix(y)

print("Starting Calculation")
start_time = time.time()
start_calculation()
end_time = time.time()

# Calculate the duration
duration = (end_time - start_time) * 1000
print("duration(ms): ",duration)
# prod = read_offset_prod_matrix()
mmio.read(0x4000)

output_matrix = [[0 for _ in range(16)] for _ in range(16)]
for i in range(16):
    for j in range(16):
        # offset for input a
        offset = offset_prod + (i * 16 + j) * 4

        # Get the value from the offset 
        value = mmio.read(offset)

        # Write the value to the calculated offset
        mmio.write(offset, value)

        # store value
        output_matrix[i][j] = value
        

print()
print("RESULT:")
print_matrix(output_matrix)

4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4

4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
4 4 4 4

In [145]:
# Initialize a list to store the duration of each calculation
# this will be used for the report later 
calculation_times = []


for i in range(10): 

    a, b = generate_random_matrices(16, 16)

    write_matrix_a(a)
    write_matrix_b(b)
    
    expected_value = matrix_multf(a,b)
    print(f"Starting Calculation {i+1}")

    matrix_mult_ip = overlay.matrix_mult_0
    start_time = time.time()
    matrix_mult_ip.register_map.CTRL.AP_START = 1
 
    # Wait for calculation to finish
    while matrix_mult_ip.register_map.CTRL.AP_IDLE == 0: 
        pass

    end_time = time.time()

    duration = (end_time - start_time) * 1000
    calculation_times.append(duration)
    
    print("duration(ms): ",duration)
    output_matrix = [[0 for _ in range(16)] for _ in range(16)]
    for i in range(16):
        for j in range(16):
            # offset for input a
            offset = offset_prod + (i * 16 + j) * 4

            # Get the value from the offset 
            value = mmio.read(offset)

            # Write the value to the calculated offset
            mmio.write(offset, value)

            # store value
            output_matrix[i][j] = value
    
#     print_matrix(prod)
#     print()
#     print_matrix(expected_value)
    
    if np.allclose(expected_value,output_matrix): 
        print(f"Calculation {i+1} Passed!")
    else: 
       print(f"Calculation {i+1} Failed!") 
    

# Calculate the average calculation time
average_time = sum(calculation_times) / len(calculation_times)

# Print the report
print("\nCalculation Times Report")
for i, duration in enumerate(calculation_times):
    print(f"Calculation {i+1}: {duration} ms")
print(f"Average Calculation Time: {average_time} ms")

Starting Calculation 1
duration(ms):  0.3275871276855469
Calculation 16 Passed!
Starting Calculation 2
duration(ms):  0.30612945556640625
Calculation 16 Passed!
Starting Calculation 3
duration(ms):  0.3159046173095703
Calculation 16 Passed!
Starting Calculation 4
duration(ms):  0.3330707550048828
Calculation 16 Passed!
Starting Calculation 5
duration(ms):  0.30994415283203125
Calculation 16 Passed!
Starting Calculation 6
duration(ms):  0.3159046173095703
Calculation 16 Passed!
Starting Calculation 7
duration(ms):  0.33545494079589844
Calculation 16 Passed!
Starting Calculation 8
duration(ms):  0.3108978271484375
Calculation 16 Passed!
Starting Calculation 9
duration(ms):  0.3006458282470703
Calculation 16 Passed!
Starting Calculation 10
duration(ms):  0.3325939178466797
Calculation 16 Passed!

Calculation Times Report
Calculation 1: 0.3275871276855469 ms
Calculation 2: 0.30612945556640625 ms
Calculation 3: 0.3159046173095703 ms
Calculation 4: 0.3330707550048828 ms
Calculation 5: 0.3099